In [11]:
import os
import joblib
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score

# ✅ Load dataset
## Load your dataset
df = pd.read_parquet("data/fraud_data.parquet")
# ✅ Features and label
target = "Class"
X_full = df.drop(columns=[target])
y = df[target]

# ✅ Define feature subsets
# Drop columns with too many unique values (e.g., more than 100)
for col in X_full.select_dtypes(include="object").columns:
    if X_full[col].nunique() > 100:
        print(f"Dropping column: {col} (unique: {X_full[col].nunique()})")
        X_full = X_full.drop(columns=col)

X_full = pd.get_dummies(X_full)  # One-hot encode all object-type columns
all_features = X_full.columns.tolist()
os.makedirs("artifacts", exist_ok=True)  # Create directory if it doesn't exist
joblib.dump(X_full.columns.tolist(), "artifacts/feature_names.pkl")

half_features = all_features[:len(all_features)//2]
top_features = all_features[10:20]  # Example: features ranked important by domain or feature importance

feature_sets = {
    "all_features": all_features,
    "half_features": half_features,
    "top_10_features": top_features
}

# ✅ Define models and hyperparameters
models = {
    "LogisticRegression": {
        "model": LogisticRegression(max_iter=500),
        "params": {"C": [0.1, 1, 10]}
    },
    "RandomForest": {
        "model": RandomForestClassifier(),
        "params": {"n_estimators": [50, 100], "max_depth": [3, 5]}
    },
    "SVC": {
        "model": SVC(),
        "params": {"C": [0.1, 1], "kernel": ["linear", "rbf"]}
    }
}


for exp_id in range(1, 11):
    experiment_name = "Fraud_Detection_Comparison"  # Define the name as a string
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    experiment = mlflow.set_experiment(experiment_name)  # Set and get experiment object
    experiment_id = experiment.experiment_id  # Extract the ID

    for feature_set_name, selected_features in feature_sets.items():
        X = X_full[selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=exp_id)

        for model_name, config in models.items():
            model = config["model"]
            param_grid = config["params"]

            grid = GridSearchCV(model, param_grid, cv=3, scoring="accuracy")
            grid.fit(X_train, y_train)

            best_model = grid.best_estimator_
            y_pred = best_model.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred, zero_division=0)
            rec = recall_score(y_test, y_pred, zero_division=0)
            f1 = f1_score(y_test, y_pred, zero_division=0)  # ✅ new
            
            with mlflow.start_run(run_name=f"{model_name}_{feature_set_name}"):
                mlflow.log_param("model", model_name)
                mlflow.log_param("feature_set", feature_set_name)
                mlflow.log_params(grid.best_params_)
                mlflow.log_metric("accuracy", acc)
                mlflow.log_metric("precision", prec)
                mlflow.log_metric("recall", rec)
                mlflow.sklearn.log_model(best_model, artifact_path="model")

                print(f"✅ Run logged for {model_name} with {feature_set_name} in {experiment_name}")
                print("🚀 Run ID:", mlflow.active_run().info.run_id)


Dropping column: TransactionID (unique: 10000)


2025/07/26 09:46:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/26 09:46:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run logged for LogisticRegression with all_features in Fraud_Detection_Comparison
🚀 Run ID: 02b0be6a02d14af58d424825e39fbea7
🏃 View run LogisticRegression_all_features at: http://127.0.0.1:5000/#/experiments/865794027230414114/runs/02b0be6a02d14af58d424825e39fbea7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/865794027230414114


2025/07/26 09:46:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/26 09:46:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run logged for RandomForest with all_features in Fraud_Detection_Comparison
🚀 Run ID: 4e14ba0ecc4b4f699f4c7dbbf8fd9ff6
🏃 View run RandomForest_all_features at: http://127.0.0.1:5000/#/experiments/865794027230414114/runs/4e14ba0ecc4b4f699f4c7dbbf8fd9ff6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/865794027230414114


...

✅ Run logged for LogisticRegression with top_10_features in Fraud_Detection_Comparison
🚀 Run ID: 5993360bb554416da62b9b595775d67b
🏃 View run LogisticRegression_top_10_features at: http://127.0.0.1:5000/#/experiments/865794027230414114/runs/5993360bb554416da62b9b595775d67b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/865794027230414114


2025/07/26 09:55:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/26 09:55:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run logged for RandomForest with top_10_features in Fraud_Detection_Comparison
🚀 Run ID: 8367310345f2407aa2ee94cbb1c45d22
🏃 View run RandomForest_top_10_features at: http://127.0.0.1:5000/#/experiments/865794027230414114/runs/8367310345f2407aa2ee94cbb1c45d22
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/865794027230414114


2025/07/26 09:55:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/26 09:55:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run logged for SVC with top_10_features in Fraud_Detection_Comparison
🚀 Run ID: 2f080b7d8ddb4b1f9e7434336e4cad97
🏃 View run SVC_top_10_features at: http://127.0.0.1:5000/#/experiments/865794027230414114/runs/2f080b7d8ddb4b1f9e7434336e4cad97
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/865794027230414114
